In [1]:
import numpy as np
import pandas as pd
import hopsworks
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [2]:
# Login to Hopsworks
project = hopsworks.login()

# Retrieve Feature Store
fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Multiple projects found. 

	 (1) quickstart_shared
	 (2) marco



Enter project to access:  2



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/397461
Connected. Call `.close()` to terminate connection gracefully.


In [95]:
# Retrieve Feature Group
port_embeddings_fg = fs.get_or_create_feature_group(
    name='port_embeddings',
    version=1,
)

# Retrieve Feature Groups
port_distance_fg = fs.get_or_create_feature_group(
    name='port_distances',
    version=1,
)

In [96]:
query = port_distance_fg.select_all().join(port_embeddings_fg.select_all(), left_on=["source_node_id"], right_on=["node_id"], prefix='origin_')

In [97]:
df = query.read(read_options={'use_hive':True})

Error: Reading data from Hopsworks, using Hive           


DatabaseError: Execution failed on sql: WITH right_fg0 AS (SELECT *
FROM (SELECT `fg1`.`source_node_id` `source_node_id`, `fg1`.`target_node_id` `target_node_id`, `fg1`.`hop_node_ids` `hop_node_ids`, `fg1`.`costs` `costs`, `fg1`.`hops` `hops`, `fg1`.`distance_km` `distance_km`, `fg1`.`timestamp_added` `timestamp_added`, `fg1`.`source_node_id` `join_pk_source_node_id`, `fg1`.`target_node_id` `join_pk_target_node_id`, `fg1`.`timestamp_added` `join_evt_timestamp_added`, `fg0`.`node_id` `origin_node_id`, `fg0`.`embedding` `origin_embedding`, `fg0`.`timestamp_added` `origin_timestamp_added`, RANK() OVER (PARTITION BY `fg1`.`source_node_id`, `fg1`.`timestamp_added` ORDER BY `fg0`.`timestamp_added` DESC) pit_rank_hopsworks
FROM `marco_featurestore`.`port_distances_1` `fg1`
INNER JOIN `marco_featurestore`.`port_embeddings_1` `fg0` ON `fg1`.`source_node_id` = `fg0`.`node_id` AND `fg1`.`timestamp_added` >= `fg0`.`timestamp_added`) NA
WHERE `pit_rank_hopsworks` = 1) (SELECT `right_fg0`.`source_no